### Demo Preparation Notebook

**Please Note**: This notebook is NOT intended to be used as learning materials. To gain
a thorough understanding of the DataJoint Element for Calcium Imaging, please
see the [`tutorial`](./tutorial.ipynb) notebook.

In [ ]:
import datetime
import suite2p
from tutorial_pipeline import *

In [ ]:
subject.Subject.insert1(
    dict(
        subject='subject1',
        subject_birth_date='2023-01-01',
        sex='U',
    )
)

In [ ]:
Equipment.insert1(dict(device="Mesoscope1",
                       modality=""))

In [ ]:
session_key = dict(subject='subject1', 
                   session_datetime=datetime.datetime(2023, 5, 11, 12, 00, 00))

In [ ]:
session.Session.insert1(session_key)

session.SessionDirectory.insert1(
    dict(
        session_key, 
        session_dir='subject1/session1'
    )
)

In [ ]:
scan.Scan.insert1(
    dict(
        session_key,
        scan_id=0,
        device="Mesoscope1",
        acq_software='ScanImage',
    )
)

In [ ]:
scan.ScanInfo.populate(display_progress=True)

In [ ]:
suite2p_params = suite2p.default_ops()
suite2p_params['nonrigid']=False

imaging.ProcessingParamSet.insert_new_params(
    processing_method="suite2p",
    paramset_idx=0,
    params=suite2p_params,
    paramset_desc='Default parameter set for suite2p'
)

In [ ]:
imaging.ProcessingTask.insert1(
    dict(
        session_key,
        scan_id=0,
        paramset_idx=0,
        task_mode='load', # load or trigger
        processing_output_dir='subject1/session1/suite2p',
    )
)

imaging.Processing.populate(display_progress=True)

In [ ]:
imaging.Curation.insert1(
    dict(
        session_key,
        scan_id=0,
        paramset_idx=0,
        curation_id=0,
        curation_time=datetime.datetime(2023, 5, 11, 12, 00, 00),
        curation_output_dir='subject1/session1/suite2p',
        manual_curation=False,
    )
)

imaging.MotionCorrection.populate(display_progress=True)
imaging.Segmentation.populate(display_progress=True)
imaging.Fluorescence.populate(display_progress=True)
imaging.Activity.populate(display_progress=True)
imaging_report.ScanLevelReport.populate(display_progress=True)
imaging_report.TraceReport.populate(display_progress=True)

### Drop schemas

- Schemas are not typically dropped in a production workflow with real data in it.
- At the developmental phase, it might be required for the table redesign.
- When dropping all schemas is needed, the following is the dependency order.

In [ ]:
def drop_databases(databases):
    import pymysql.err
    conn = dj.conn()

    with dj.config(safemode=False):
        for database in databases:
            schema = dj.Schema(f'{dj.config["custom"]["database.prefix"]}{database}')
            while schema.list_tables():
                for table in schema.list_tables():
                    try:
                        conn.query(f"DROP TABLE `{schema.database}`.`{table}`")
                    except pymysql.err.OperationalError:
                        print(f"Can't drop `{schema.database}`.`{table}`. Retrying...")
            schema.drop()

# drop_databases(databases=['analysis', 'trial', 'event', 'imaging_report', 'imaging', 'scan', 'session', 'subject', 'lab', 'reference'])